In [1]:
import os
# 以相對路徑變更工作目錄
os.chdir(r'C:/Users/PEK/test0821-1') 

In [2]:
import pickle
import sys
import numpy as np
import os
import scipy.signal as sg
import wfdb
from tqdm import tqdm
import numpy as np
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from scipy.signal import resample
import glob
import pywt
import pandas as pd
import neurokit2 as nk
import csv
from collections import Counter
import warnings
from IPython.display import display
import shutil
import posixpath
import pyhrv
import json
from vectorizedsampleentropy import vectsampen as vse
import math
import heartpy as hp
from statsmodels.tsa.ar_model import AR
import pyhrv.tools as tools
import glob
from sklearn.metrics import r2_score
import collections
from math import sqrt
from sklearn.metrics import mean_squared_error
import scipy.signal
from biosppy.signals import ecg
import gc
gc.enable()
pd.set_option('display.max_colwidth', None)
fs = 125
base_dir = "dataset"
sampling_rate = 360
invalid_labels = ['|', '~', '!', '+', '[', ']', '"', 'x']  # non-beat labels
before = 90
after = 110
tol = 0.05
cpuCount = os.cpu_count() 
import psutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import sys
import rpy2.rinterface
from function_all import *

Using TensorFlow backend.


In [3]:
# using R code

latex2exp = importr('latex2exp')
nonlinearTseries = importr('nonlinearTseries')
plot3D = importr('plot3D')
raster = importr('raster')
dplyr = importr('dplyr')
lubridate = importr('lubridate')
# Defining the R script and loading the instance in Python
r = robjects.r
r['source']('HRV grid counting.r')
# Loading the function we have defined in R.
ECG_grid_func_r = robjects.globalenv['ECGgrid']
# # Load in R libraries 
pandas2ri.activate()

In [4]:
# Define HRV input parameters
# Time Domain Settings
settings_time = {
    'threshold': 50,            # Computation of NNXX/pNNXX with 50 ms threshold -> NN50 & pNN50
    'plot': True,               # If True, plots NNI histogram
    'binsize': 7.8125           # Binsize of the NNI histogram
}

# Frequency Domain Settings
settings_welch = {
    'nfft': 2 ** 12,            # Number of points computed for the FFT result
    'detrend': True,            # If True, detrend NNI series by subtracting the mean NNI
    'window': 'hanning'         # Window function used for PSD estimation
}

settings_lomb = {
    'nfft': 2**8,               # Number of points computed for the Lomb PSD
    'ma_size': 5                # Moving average window size
}

settings_ar = {
    'nfft': 2**12,              # Number of points computed for the AR PSD
    'order': 32                 # AR order
}

# Nonlinear Parameter Settings
settings_nonlinear = {
    'short': [4, 16],           # Interval limits of the short term fluctuations
    'long': [17, 64],           # Interval limits of the long term fluctuations
    'dim': 2,                   # Sample entropy embedding dimension
    'tolerance': None           # Tolerance distance for which the vectors to be considered equal (None sets default values)
}

In [5]:
model = load_model(os.path.join("", "model_focalloss.h5"),
        custom_objects={"focal_loss": categorical_focal_loss(gamma=2),"f1": f1})
model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
# input raw ECG data
df = pd.read_csv('ECG 10min example.csv')
df

,I
0,130.0
1,128.0
2,127.0
3,126.0
4,125.0
...,...
74995,105.0
74996,110.0
74997,111.0
74998,110.0


In [7]:
II_data = df['I']
del  df
hrv_update = hp.preprocessing.scale_data(II_data, lower=0, upper=200)
del II_data
hrv1=pd.DataFrame(['NA','NA','NA','NA','NA','NA','NA','NA'])
h0=hrv1.transpose()
h0.columns =['aFdP', 'fFdP', 'ARerr', 'DFA.Alpha.1', 'Mean.rate',
                       'Poincar..SD2', 'shannEn', 'LF.HF.ratio.LombScargle']                     
hrv1=hrv1.transpose()
final1 = hrv_update
v0=0
v1=hrv_update.shape[0]
step0=37500
bin0=np.append(np.arange(v0, v1, step = step0), v1)
index = [0]
new_bin0 = np.delete(bin0, index)
check=[]

for i in new_bin0: 
        try:
            warnings.filterwarnings("ignore")
            index = np.append(np.arange((i/step0-1)*step0+1, i, step = 1),i)
            index =index.astype(int)-1
            hrdata1 = final1[index]
            hrdata1 = hrdata1[~pd.isnull(hrdata1)]
            hrdata1 = np.array(hrdata1)  
            x1_test,x2_test = wave_to_newtest_input_data_form1(hrdata1,show=False)
            new_test=np.argmax(model.predict([x1_test,x2_test], batch_size=1024, verbose=0), axis=-1)                    
            result = np.array([Counter(new_test)[0]/len(new_test),Counter(new_test)[1]/len(new_test),Counter(new_test)[2]/len(new_test),Counter(new_test)[3]/len(new_test)])
            r1 = np.reshape(result, (1, 4))
            check.append(r1.item(0)) 
            data1 = normal_ecg_transfer(hrdata1,new_test,fs)
            r_from_pd_df = ro.conversion.py2rpy(data1)
            hrvpart0= hrvtransform2_only_normal_ECG_filter_hr(hrdata1,new_test,fs,hrv1,settings_time,settings_welch,settings_ar,settings_lomb,settings_nonlinear)
            hrvpart = pd.DataFrame(data=ECG_grid_func_r(r_from_pd_df),columns = ['gcount','sgridTAU'])
            hrvpart['time_min'] = i/7500
            warnings.filterwarnings("ignore")
            # remove dataframe 2 indices
            hrvpart0.reset_index(drop=True, inplace=True)
            # remove dataframe 2 indices
            hrvpart.reset_index(drop=True, inplace=True)
            hrvpart = pd.concat([hrvpart0, hrvpart], axis=1)
            h0 = h0.append(pd.DataFrame(data = hrvpart), ignore_index=True)

        except:
            continue
h0 = h0.iloc[1:]

In [8]:
# output 10 variables
h0

,aFdP,fFdP,ARerr,DFA.Alpha.1,Mean.rate,Poincar..SD2,shannEn,LF.HF.ratio.LombScargle,gcount,sgridTAU,time_min
1,-0.965319,-0.965606,0.194097,0.597574,80.9225,0.162607,6.05918,0.521876,32.2500,-1.3750,5.0
2,-0.969526,-0.969573,0.174346,0.644713,84.5498,0.149982,5.89388,0.417417,37.3125,1.0625,10.0
